# Alphalens Analysis

### Loading libraries

In [19]:
# Numerical Computing
import numpy as np

# Data Manipulation
import pandas as pd
import pandas_datareader.data as web

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Model Interpreter
import shap

# Warnings
import warnings

# Path, OS & Time
import os, sys
from time import time
from pathlib import Path

# StatsModel
import statsmodels.api as sm

# Scikit-Learn
from sklearn.preprocessing import scale
from sklearn.feature_selection import mutual_info_regression

# SciPy
from scipy.stats import spearmanr

# Gradient Booster
import lightgbm as lgb

In [20]:
from utils import MultipleTimeSeriesCV

In [21]:
sys.path.insert(1, os.path.join(sys.path[0], '..'))

In [22]:
idx= pd.IndexSlice

sns.set_style('whitegrid')

warnings.filterwarnings('ignore')

In [23]:
results_path = Path('results')

if not results_path.exists():
    results_path.mkdir()

In [29]:
# DATA_STORE = Path('..', 'data', 'assets.h5')

### Getting Trade Prices

In [32]:
def get_trade_prices(tickers):
    return (pd.read_hdf(DATA_STORE, 'quandl/wiki/prices')
              .loc[idx['2006':'2017', tickers], 'adj_open']
              .unstack('ticker')
              .sort_index()
            .shift(-1)
            .tz_localize('UTC'))

In [ ]:
# trade_prices = get_trade_prices(tickers)

In [ ]:
# trade_prices.info()

### Loading Factors

In [36]:
# factors = (pd.concat([pd.read_hdf('data.h5', 'factors/common'),
#                       pd.read_hdf('data.h5', 'factors/formulaic')
#                       .rename(columns=lambda x: f'alpha_{int(x):03}')],
#                      axis=1)
#            .dropna(axis=1, thresh=100000)
#            .sort_index())

In [38]:
# factors.info()

In [40]:
# tickers = factors.index.get_level_values('ticker').unique()

In [42]:
# alpha = 'alpha_054'

In [44]:
# factor = (factors[alpha]
#           .unstack('ticker')
#           .stack()
#           .tz_localize('UTC', level='date')
#           .sort_index())

### Generating Alphalens Input Data

In [56]:
# factor_data = utils.get_clean_factor_and_forward_returns(factor=factor,
#                                                    prices=trade_prices,
#                                                    quantiles=5,
#                                                    max_loss=0.35,
#                                                    periods=(1, 5, 10)).sort_index()
# factor_data.info()

### Computing Metrics

In [59]:
# mean_quant_ret_bydate, std_quant_daily = perf.mean_return_by_quantile(
#     factor_data,
#     by_date=True,
#     by_group=False,
#     demeaned=True,
#     group_adjust=False,
# )

# mean_quant_rateret_bydate = mean_quant_ret_bydate.apply(
#     rate_of_return,
#     base_period=mean_quant_ret_bydate.columns[0],
# )

# compstd_quant_daily = std_quant_daily.apply(std_conversion,
#                                             base_period=std_quant_daily.columns[0])

# alpha_beta = perf.factor_alpha_beta(factor_data,
#                                     demeaned=True)

# mean_ret_spread_quant, std_spread_quant = perf.compute_mean_returns_spread(
#     mean_quant_rateret_bydate,
#     factor_data["factor_quantile"].max(),
#     factor_data["factor_quantile"].min(),
#     std_err=compstd_quant_daily,
# )

In [61]:
# mean_ret_spread_quant.mean().mul(10000).to_frame('Mean Period Wise Spread (bps)').join(alpha_beta.T).T

### Plotting Spread & Cumulative Returns

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize=(20, 5))

mean_quant_ret, std_quantile = mean_return_by_quantile(factor_data,
                                                       by_group=False,
                                                       demeaned=True)

mean_quant_rateret = mean_quant_ret.apply(rate_of_return, axis=0,
                                          base_period=mean_quant_ret.columns[0])

plot_quantile_returns_bar(mean_quant_rateret, ax=axes[0])


factor_returns = perf.factor_returns(factor_data)

title = "Factor Weighted Long/Short Portfolio Cumulative Return (1D Period)"
plotting.plot_cumulative_returns(factor_returns['1D'],
                                 period='1D',
                                 freq=pd.tseries.offsets.BDay(),
                                 title=title,
                                 ax=axes[1])

plotting.plot_cumulative_returns_by_quantile(mean_quant_ret_bydate['1D'],
                                             freq=pd.tseries.offsets.BDay(),
                                             period='1D',
                                             ax=axes[2])
fig.tight_layout();